In [1]:
import os
import sys
import re
import time
from pathlib import Path
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import warnings
import logging
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt

sys.path.append('../python')
warnings.filterwarnings('default')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

DATA_DIR = Path('../../raw_data/cpc')

import api

/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
meetings_meta_df = pd.read_csv("../../intermediate_data/cpc/meetings_metadata.csv")

In [3]:
PROMPT = """
The following extracted PDF text contains the agenda for a LA City Planning Commission meeting. 

For each agenda item, return a summary in the following format:

ITEM NO: <agenda item number>
TITLE: <title of agenda item>
SUMMARY: <short summary of agenda item>

Separate each response by "------"

AGENDA:

{}
"""

In [4]:
# Use ChatGPT to separate agenda items

for idx, row in meetings_meta_df.loc[meetings_meta_df['has_all']].iterrows():
    year = row['year']
    date = row['date']
    print(date, end='... ')
    try:
        with open(f"../../intermediate_data/cpc/{year}/{date}/agenda-manual-override.txt", 'r') as f:
            text = f.read()
    except:
        with open(f"../../intermediate_data/cpc/{year}/{date}/agenda.txt", 'r') as f:
            text = f.read()
    prompt = PROMPT.format(text)
    message, score = api.get_response(prompt)
    output_file = f"../../intermediate_data/cpc/{year}/{date}/agenda-item-summaries.txt"
    with open(output_file, 'w') as f:
        f.write(message)

2018-05-10... 2018-05-23... 2018-06-14... 2018-07-12... 2018-07-26... 2018-08-09... 2018-08-23... 2018-09-13... 2018-09-27... 2018-10-11... 2018-10-25... 2018-11-08... 2018-11-29... 2018-12-13... 2018-12-20... 2019-01-10... 2019-01-24... 2019-02-14... 2019-02-28... 2019-03-14... 2019-03-28... 2019-04-11... 2019-05-09... 2019-05-23... 2019-06-13... 2019-06-27... 2019-07-11... 2019-07-25... 2019-08-08... 2019-08-22... 2019-09-12... 2019-09-26... 2019-10-10... 2019-10-24... 2019-11-14... 2019-11-21... 2019-12-12... 2019-12-19... 2020-01-09... 2020-01-23... 2020-02-13... 2020-03-12... 2020-04-23... 2020-05-14... 2020-05-28... 2020-06-04... 2020-06-11... 2020-06-25... 2020-07-09... 2020-07-23... 2020-08-13... 2020-08-27... 2020-09-10... 2020-09-17... 2020-09-24... 2020-10-08... 2020-10-15... 2020-10-22... 2020-11-05... 2020-11-19... 2020-12-03... 2020-12-10... 2020-12-17... 2021-01-14... 2021-01-28... 2021-02-11... 2021-02-18... 2021-02-25... 2021-03-11... 2021-03-25... 2021-04-08... 2021-0